In [118]:
import numpy as np
import pandas as pd

In [119]:
application_record = pd.read_csv("application_record.csv")
application_record = application_record[["ID", "FLAG_OWN_CAR", "FLAG_OWN_REALTY", "CNT_CHILDREN", "AMT_INCOME_TOTAL", "NAME_INCOME_TYPE", "NAME_EDUCATION_TYPE", "NAME_FAMILY_STATUS", "NAME_HOUSING_TYPE", "DAYS_BIRTH", "DAYS_EMPLOYED", "CNT_FAM_MEMBERS"]]

application_record["FLAG_OWN_CAR"] = application_record["FLAG_OWN_CAR"].map(lambda x: x == "Y")
application_record["FLAG_OWN_REALTY"] = application_record["FLAG_OWN_REALTY"].map(lambda x: x == "Y")

INCOME_TYPE = ["Working", "Commercial associate", "Pensioner", "State servant", "Student"]
EDUCATION_TYPE = ["Secondary / secondary special", "Higher education", "Incomplete higher", "Lower secondary", "Academic degree"]
FAMILY_STATUS = ["Married", "Single / not married", "Civil marriage", "Separated, Widow"]
HOUSING_TYPE = ["House / apartment", "With parents", "Municipal apartment", "Rented apartment", "Office apartment"]

application_record["NAME_INCOME_TYPE"] = pd.Categorical(application_record["NAME_INCOME_TYPE"], categories=INCOME_TYPE, ordered=True)
application_record["NAME_EDUCATION_TYPE"] = pd.Categorical(application_record["NAME_EDUCATION_TYPE"], categories=EDUCATION_TYPE, ordered=True)
application_record["NAME_FAMILY_STATUS"] = pd.Categorical(application_record["NAME_FAMILY_STATUS"], categories=FAMILY_STATUS, ordered=True)
application_record["NAME_HOUSING_TYPE"] = pd.Categorical(application_record["NAME_HOUSING_TYPE"], categories=HOUSING_TYPE, ordered=True)

application_record

,ID,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,CNT_FAM_MEMBERS
0,5008804,True,True,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,2.0
1,5008805,True,True,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,2.0
2,5008806,True,True,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,2.0
3,5008808,False,True,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1.0
4,5008809,False,True,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
438552,6840104,False,True,0,135000.0,Pensioner,Secondary / secondary special,NaN,House / apartment,-22717,365243,1.0
438553,6840222,False,False,0,103500.0,Working,Secondary / secondary special,Single / not married,House / apartment,-15939,-3007,1.0
438554,6841878,False,False,0,54000.0,Commercial associate,Higher education,Single / not married,With parents,-8169,-372,1.0
438555,6842765,False,True,0,72000.0,Pensioner,Secondary / secondary special,Married,House / apartment,-21673,365243,2.0


In [120]:
credit_record = pd.read_csv("credit_record.csv")
credit_record

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C
...,...,...,...
1048570,5150487,-25,C
1048571,5150487,-26,C
1048572,5150487,-27,C
1048573,5150487,-28,C


In [121]:
def group_is_good(group):
    return group["STATUS"].isin(['C', 'X']).sum() > len(group) / 2

is_good_map = credit_record.groupby("ID").apply(group_is_good, include_groups=False)

In [122]:
application_record["IS_GOOD"] = application_record["ID"].map(is_good_map).fillna(True)
application_record

C:\Users\moham\AppData\Local\Temp\ipykernel_14836\3859961568.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  application_record["IS_GOOD"] = application_record["ID"].map(is_good_map).fillna(True)


,ID,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,CNT_FAM_MEMBERS,IS_GOOD
0,5008804,True,True,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,2.0,True
1,5008805,True,True,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,2.0,True
2,5008806,True,True,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,2.0,True
3,5008808,False,True,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1.0,True
4,5008809,False,True,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
438552,6840104,False,True,0,135000.0,Pensioner,Secondary / secondary special,NaN,House / apartment,-22717,365243,1.0,True
438553,6840222,False,False,0,103500.0,Working,Secondary / secondary special,Single / not married,House / apartment,-15939,-3007,1.0,True
438554,6841878,False,False,0,54000.0,Commercial associate,Higher education,Single / not married,With parents,-8169,-372,1.0,True
438555,6842765,False,True,0,72000.0,Pensioner,Secondary / secondary special,Married,House / apartment,-21673,365243,2.0,True


In [123]:
application_record[["NAME_INCOME_TYPE", "IS_GOOD"]].groupby("NAME_INCOME_TYPE").mean()

C:\Users\moham\AppData\Local\Temp\ipykernel_14836\2990562632.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  application_record[["NAME_INCOME_TYPE", "IS_GOOD"]].groupby("NAME_INCOME_TYPE").mean()


,IS_GOOD
NAME_INCOME_TYPE,
Working,0.960085
Commercial associate,0.959556
Pensioner,0.959983
State servant,0.960924
Student,0.823529


In [124]:
application_record[["IS_GOOD"]].mean()

IS_GOOD    0.96001
dtype: float64